<strong>Dependencies</strong>: MeshPy, PyVTK, trimesh

<strong>Useful links</strong>:
 - TetGen : http://wias-berlin.de/software/tetgen/1.5/doc/manual/manual006.html
 - MeshPy : https://documen.tician.de/meshpy/tri-tet.html#meshpy.tet.MeshInfo.adjacent_elements
 
<strong>Useful research</strong>:
 - SRF BluePrint : http://i.cs.hku.hk/~wenping/allhex.pdf
 - QuadCover : 
 - CubeCover : http://tinyurl.com/zd7yj6j
 - Principal Curvature : http://gfx.cs.princeton.edu/pubs/_2004_ECA/curvpaper.pdf

In [1]:
import meshpy.tet
import numpy as np
import trimesh

## Generate Tetrahedral Mesh

In [2]:
tri_mesh = trimesh.load_mesh('../data/cylinder.stl')

# Define MeshPy options!
opt = meshpy.tet.Options(switches='pq', edgesout=True, facesout=True, neighout=True)

mesh_info = meshpy.tet.MeshInfo()
mesh_info.set_points(tri_mesh.vertices)
# Convert face data from np.int64 to int...
faces = [list(map(lambda x: int(x), i)) for i in tri_mesh.faces]
mesh_info.set_facets(faces)
tet_mesh = meshpy.tet.build(mesh_info, opt, max_volume=10)

# Output tetrahedral mesh
tet_mesh.write_vtk("../data/test.vtk")

## Compute surface normals

#### Some useful information

 - Internal tetrahedrons have 4 neighbors (one on each face) - the neighbor 4-tuple is the indices of the neighboring tets.

 - The first neighbor of tetrahedron 'i' is oppposite to the first corner of tetrahedron 'i', and so on.

 - An index of -1 indicates there is no neighbor.
 
 - Tetrahedral face orientation:
 
    - 3 - 0 - 1
 
    - 1 - 2 - 3
   
    - 3 - 2 - 0  (unusual)
   
    - 2 - 1 - 0  (unusual)
   

In [20]:
surface_faces = []
for i, tet in enumerate(tet_mesh.elements):
    neighbors = list(tet_mesh.neighbors[i])
    # TODO(aidan) This overlooks case of multiple surface faces...
    try:
        non_surface_vtx = tet[neighbors.index(-1)]
        tet_cpy = tet.copy()
        tet_cpy.remove(non_surface_vtx)
        if neighbors.index(-1) in (1, 3):
            tet_cpy.reverse()
        surface_faces.append(tet_cpy)
    except ValueError:
        continue
        
# Compute normal
def compute_normal(face):
    return np.cross(np.array(tet_mesh.points[face[1]]) - np.array(tet_mesh.points[face[0]]),
                    np.array(tet_mesh.points[face[2]]) - np.array(tet_mesh.points[face[1]]))

def compute_avg(face):
    return (np.array(tet_mesh.points[face[0]])
          + np.array(tet_mesh.points[face[1]])
          + np.array(tet_mesh.points[face[2]])) / 3

face_normals = [compute_normal(face) for face in surface_faces]
face_center = [compute_avg(face) for face in surface_faces]

### Visualize normals

In [21]:
from mpl_toolkits.mplot3d import proj3d
import mpl_toolkits.mplot3d as a3
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure(num=None, figsize=(12, 10), dpi=80)
ax = fig.add_subplot(111, projection='3d')

for i, f in enumerate(surface_faces):
    ax.quiver(face_center[i][0], face_center[i][1], face_center[i][2], face_normals[i][0], face_normals[i][1], face_normals[i][2])

plt.show()

## Compute surface curvatures

In [22]:
# Compute one-ring neighborhoods
dedup_vertices = set(list(np.concatenate(surface_faces)))
surrounding_faces = {}
for i, v in enumerate(dedup_vertices):
    surrounding_faces[v] = list(filter(lambda f: f[1][0] == v or f[1][1] == v or f[1][2] == v, zip(range(0, len(surface_faces)), surface_faces)))

In [29]:
# Compute vertex normals
from numpy import vstack

def get_vertex_norm(face):
    return vstack([face_normals[ind] for ind, f in faces]).mean(axis=0)

v_norms = {}
for i, v in enumerate(dedup_vertices):
    faces = surrounding_faces[v]
    v_norms[v] = get_vertex_norm(faces)

array([ 0.82856348, -0.01222879, -1.33190271])

In [24]:
# Plot vertex normals
from mpl_toolkits.mplot3d import proj3d
import mpl_toolkits.mplot3d as a3
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure(num=None, figsize=(12, 10), dpi=80)
ax = fig.add_subplot(111, projection='3d')

for i in dedup_vertices:
    i = int(i)
    ax.quiver(tet_mesh.points[i][0], tet_mesh.points[i][1], tet_mesh.points[i][2], v_norms[i][0], v_norms[i][1], v_norms[i][2])

plt.show()

In [34]:
""" COMPUTE CURVATURE """

from numpy import linalg, transpose, dot, cross, multiply
from numpy.linalg import eig, norm


def length(v):
    return linalg.norm(v)

def Matrix(arr_of_3):
    m = np.zeros((3, 3))
    m[0][0] = arr_of_3[0]
    m[1][1] = arr_of_3[1]
    m[2][2] = arr_of_3[2]
    return m

def get_vertices_from_faces(v):
    ring = list(set(np.concatenate([vert for ind, vert in surrounding_faces[v]])))
    ring.remove(v)
    return ring

def get_complement_basis(normal):
    if normal[2] 

max_curv = []
min_curv = []

for face in surface_faces:
    
    SFT = [] 
    
    e0 = np.array(face[2]) - np.array(face[1])
    e1 = np.array(face[0]) - np.array(face[2])
    e2 = np.array(face[1]) - np.array(face[0])

    n0 = np.array(v_norms[0])
    n1 = np.array(v_norms[1])
    n2 = np.array(v_norms[2])
    
    
    
    
"""
# Iterate through each vertex
for i, v in enumerate(dedup_vertices):
    v = int(v)
    position = np.array(tet_mesh.points[v])
    # Vertex normal as a matrix
    n0 = np.array(v_norms[v]) / length(v_norms[v])

    # Unsorted ring
    ring = get_vertices_from_faces(v)
    
    # Calculate Mvi
    I = np.identity(3)
    Mvi = np.zeros((3, 3))

    curvature = np.zeros((len(ring),))
    Mvi = np.zeros((3, 3))

    for i, j in enumerate(ring):
        n1 = np.array(v_norms[j])
        vec = np.array(position) - np.array(tet_mesh.points[int(j)])
        # edgeAsMatrix = Matrix([vec[0], vec[1], vec[2]])
        curvature[i] = dot((2 * n0), vec) / (length(vec) ** 2)
        Mvi += curvature[i] * transpose(np.matrix(vec)) * vec

    # givens_rotation
    maxc, minc = givens_rotation(Mvi)
    max_curv.append(maxc)
    min_curv.append(minc)
    
    # Get eigenvalues and eigenvectors for Mvi
    evals, evecs = eig(Mvi)
"""

In [35]:
def gram_schmidt_process(A):
    """Perform QR decomposition of matrix A using Gram-Schmidt process."""
    (num_rows, num_cols) = np.shape(A)

    # Initialize empty orthogonal matrix Q.
    Q = np.empty([num_rows, num_rows])
    cnt = 0

    # Compute orthogonal matrix Q.
    for a in A.T:
        u = np.copy(a)
        for i in range(0, cnt):
            proj = np.dot(np.dot(Q[:, i].T, a), Q[:, i])
            u -= proj

        e = u / np.linalg.norm(u)
        Q[:, cnt] = e

        cnt += 1  # Increase columns counter.

    # Compute upper triangular matrix R.
    R = np.dot(Q.T, A)

    return (Q, R)


from math import copysign, hypot
def givens_rotation(A):
    """Perform QR decomposition of matrix A using Givens rotation."""
    (num_rows, num_cols) = np.shape(A)

    # Initialize orthogonal matrix Q and upper triangular matrix R.
    Q = np.identity(num_rows)
    R = np.copy(A)

    # Iterate over lower triangular matrix.
    (rows, cols) = np.tril_indices(num_rows, -1, num_cols)
    for (row, col) in zip(rows, cols):

        # Compute Givens rotation matrix and
        # zero-out lower triangular matrix entries.
        if R[row, col] != 0:
            (c, s) = _givens_rotation_matrix_entries(R[col, col], R[row, col])

            G = np.identity(num_rows)
            G[[col, row], [col, row]] = c
            G[row, col] = s
            G[col, row] = -s

            R = np.dot(G, R)
            Q = np.dot(Q, G.T)

    return (Q, R)

def _givens_rotation_matrix_entries(a, b):
    """Compute matrix entries for Givens rotation."""
    r = hypot(a, b)
    c = a/r
    s = -b/r

    return (c, s)

### Visualize curvature

In [36]:
from mpl_toolkits.mplot3d import proj3d
import mpl_toolkits.mplot3d as a3
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure(num=None, figsize=(12, 10), dpi=80)
ax = fig.add_subplot(111, projection='3d')

for i, v in enumerate(list(dedup_vertices)):
    i = int(i)
    v = int(v)
    maxc = [max_curv[i][0,0], max_curv[i][1,1], max_curv[i][2,2]]
    minc = [min_curv[i][0,0], min_curv[i][1,1], min_curv[i][2,2]]
    ax.quiver(tet_mesh.points[v][0], tet_mesh.points[v][1], tet_mesh.points[v][2], maxc[0], maxc[1], maxc[2])
    ax.quiver(tet_mesh.points[v][0], tet_mesh.points[v][1], tet_mesh.points[v][2], minc[0], minc[1], minc[2])

#drawmesh()

plt.show()